In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.SFI_FIM import SFIModel_gating,SFIModel_unified

In [2]:
hparams = {
    'scale':'demo',
    'name':'sfi-fim',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'filter_num':150,
    'epochs':1,
    'metrics':'auc,group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'integration':'harmony',
    'k':10,
    'save_step':0,
    'save_each_epoch':True,
    'train_embedding':True
}

device = torch.device(hparams['device'])
torch.cuda.set_device(hparams['device'])

In [5]:
vocab, loaders = prepare(hparams)

In [6]:
hparams['select'] = 'gating'
sfiModel_gating = SFIModel_gating(hparams, vocab=vocab).to(device)

In [7]:
train(sfiModel_gating, hparams, loaders[0], loaders[1], loaders[2])

training...
epoch 1 , step 200 , loss: 1.5666: : 299it [00:20, 14.77it/s]
saved model of epoch 1
testing...
1815it [00:34, 52.60it/s]
evaluation results:{'group_auc': 0.5488, 'mean_mrr': 0.2555, 'ndcg@5': 0.2644, 'ndcg@10': 0.3321}
validating...
7535it [02:20, 53.48it/s]
evaluation results:{'group_auc': 0.651, 'mean_mrr': 0.3201, 'ndcg@5': 0.3435, 'ndcg@10': 0.4007}


SFIModel_gating(
  (softmax): Softmax(dim=-1)
  (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
  (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (ReLU): ReLU()
  (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (DropOut): Dropout(p=0.2, inplace=False)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (learningToRank): Linear(in_features=64, out_features=1, bias=True)
)

In [5]:
hparams['select'] = 'unified'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [ ]:
train(sfiModel_unified, hparams, loaders[0], loaders[1], loaders[2])

In [ ]:
hparams['select'] = 'unified'
hparams['integration'] = 'gate'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [ ]:
train(sfiModel_unified, hparams, loaders[0], loaders[1], loaders[2])

In [9]:
t = SFIModel_gating(hparams, vocab).to(device)
t.cdd_size = 1

record = next(iter(loaders[1]))